# Report

Gruppe D

## Introduction and data

> REMOVE THE FOLLOWING TEXT

This section includes an introduction to the project motivation, data, and research question.
Describe the data and definitions of key variables.

It should also include some exploratory data analysis.

All of the EDA won't fit in the paper, so focus on the EDA for the response variable and a few other interesting variables and relationships.

## Methodology

> REMOVE THE FOLLOWING TEXT

This section includes a brief description of your modeling process.

Explain the reasoning for the type of model you're fitting, predictor variables considered for the model.

Additionally, show how you arrived at the final model by describing the model selection process, variable transformations (if needed), assessment of conditions and diagnostics, and any other relevant considerations that were part of the model fitting process.

Wie in der Introduction beschrieben haben wir uns aufgrund der Literartur-Recherche wie auch nach Betrachtung des Data Sets für 6 Quoten entschieden, die die Arbeitslosenquote beeinflussen könnten. 

Wie in der explorativen Analyse zu sehen ist, hat nur die Christenquote eine Korrelation mit der Arbeitslosenquote über 0,5 bzw. unter -0,5. Zwei weitere Quoten, die noch eine moderarte bis schwache Korrelation haben sind die Singlequote sowie die Migrationsquote. Daher wurde bereits an dieser Stelle beschlossen keine weiteren Quoten für die lineare Regression mit in die Auswahl aufzunehmen. Da in den Scatterplot linere Zusammenhänge erkennbar sind, wollen wir diese mit lineran Modellen versuchen zu erklären.

Für die multiple Regression wie auch die Lasso-Regression haben wir zunächst alle Variablen als Prädikatoren ausgewählt.

Zu Beginn der jeweiligen Modellierungsprozesse haben wir jeweils das Regressionsmodell ausgewählt. Der nächste Schritt war es dann die Modelle zu validieren und an die Daten anzupassen. 
Die Validierung haben wir mit der Cross-Validation umgesetzt und dabei den Mean Squared error pro Fold angeschaut.
Bei der Lasso-Regression war die Cross-Validation implementiert.

Daraufhin haben wir mit der Funktion `reg.fit` die Modelle an die Traingsdaten angepasst und somit trainiert und den Intercept mit der y-Achse sowie den beziehungsweise die Koeffizienten berechnet.

Der abschließende Schritt war die Evaluierung mit den Test-Daten. Wir haben die Modell dann anhand verschiedener Gütemaße (R^2, MSE, RMSE, MAE) bewertet.

Durch die Peer-Review haben wir das Feedback bekommen, dass wir für die Multiple Regression die Stepwise Selection nutzen könnten sowie für die Bewertung der Güte der Lasso-Regression den AIC. 

Diese grob beschriebenen Schritte wollen wir im Folgenden detaillierter erklären.

## Results

> REMOVE THE FOLLOWING TEXT

This is where you will output the final model with any relevant model fit statistics.

Describe the key results from the model.
The goal is not to interpret every single variable in the model but rather to show that you are proficient in using the model output to address the research questions, using the interpretations to support your conclusions.

Focus on the variables that help you answer the research question and that provide relevant context for the reader.


## Discussion + Conclusion


> REMOVE THE FOLLOWING TEXT

In this section you'll include a summary of what you have learned about your research question along with statistical arguments supporting your conclusions.
In addition, discuss the limitations of your analysis and provide suggestions on ways the analysis could be improved.
Any potential issues pertaining to the reliability and validity of your data and appropriateness of the statistical analysis should also be discussed here.
Lastly, this section will include ideas for future work.